Generate local predictions of data located in edgebox and compare them with the previously online created results (Just generation. Not simulation)

 Load Libraries

In [ ]:
#### Load Libraries
import numpy as np
from joblib import load
from keras.models import Model, load_model
import glob 
import  pandas as pd
import matplotlib.pyplot as plt
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
import os
from pymongo import MongoClient
import plotly.express as ex
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

DB Config

In [ ]:
#####DB Config
host= "DevEdgeV32"
port=27017

Set Parameters

In [ ]:
### set parameters
preprocessor_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/preprocessor/'
preprocessor_name='preprocessor.p'
model_dir='G:/Innovations@HELLER/DN/KI/Temperaturkompensation/New_Experiments/assets/model/'
model_name= 'model.h5'
selected_Columns=  ['t_bett', 't_spindle','t_motor', 'M8','M121', 'M127', 'M7']# So must be the input of the model
output_variable= 'welle_z'
prediction_variable= 'prediction'
window = 60
shift = 6
sampling_rate= 6

Funtion to remove t_bett from t_spindel and t_motor

In [ ]:
def remove_t_bett___(data: np.ndarray):
    result= data[:, 1:]
    result[:,0]= result[:,0] - data[:,0]
    result[:,1]= result[:,1] - data[:,0]
    return result

In [ ]:
def rounding(signals: np.ndarray):
    ##rounding t_motor to nearest integer
    signals[:,1]= np.round(signals[:,1])
    signals= np.round(signals, 2)
    return signals

def rounding2(signals: np.ndarray):
    return np.round(signals, 5)


def remove_t_bett(data: np.ndarray):
    result = data[:, 1:]
    result[:, 0] = result[:, 0] - data[:, 0]
    result[:, 1] = result[:, 1] - data[:, 0]
    return result

Function to smooth a signal

In [ ]:
def remove_peeks(dlf: pd.DataFrame, feature: str,window):
    df= dlf.copy()
    new_Feature_name= 'smoothed_' + feature
    df[new_Feature_name]= df[feature].rolling(window).mean().to_list()
    shape= df.shape
    offset= int(np.round(window/2))
    for i in range(df.shape[0] - offset):
        df.loc[i, new_Feature_name]= df.iloc[i + offset][new_Feature_name]
    df= df[df[new_Feature_name] == df[new_Feature_name]]
    return df

Function to create a data set, where each data point is a window of n steps (look window feature)

In [ ]:
### window  represents time period by each entry in the buffer
### Shift represents the jump from value to next one in the buffer
### sampling rate 
def generateDataSource(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1, sample_rate=1):
    subsequence_len= (window -1) *shift + 1
    Signal_Length = signal_input.shape[0]
    #num_samples = int((Signal_Length - (window-1) * shift))
    #num_samples = int((Signal_Length - window + 1) / shift)
    num_samples = 1 + int((Signal_Length - subsequence_len) / sample_rate)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = np.asarray([signal_input[i*sample_rate + j * shift] for j in range(0,window)])
        y[i] = signal_output[i*sample_rate + (window-1) * shift :i*sample_rate+ (window-1) * shift + output_length]
#        print('x[{}]= {}'.format(i, x[i]))
#        print('y[{}]= {}'.format(i, y[i]))
    return x, y

Set the time period to get the data

In [ ]:
#start= datetime.strptime("2022-09-20T22:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")# Trocken iso
#end= datetime.strptime ("2022-09-21T08:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
#start= datetime.strptime("2023-02-02T10:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")# Trocken iso
#end= datetime.strptime ("2023-02-02T18:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-01-31T17:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") #Messtaster mit Fräsversuch
# end= datetime.strptime ("2023-01-31T23:59:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
# start= datetime.strptime("2023-02-23T18:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000") #Messtaster mit Fräsversuch
# end= datetime.strptime ("2023-02-24T00:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
start= datetime.strptime("2023-03-01T11:00:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end= datetime.strptime ("2023-03-01T17:59:59.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

Read data from the mongo_DB database and apply preprocessing

In [ ]:
## Read raws and preprocessing data
client = MongoClient(host= host, port=port)
db = client.h4ai
event_list = db.modelLogs.find({ "date" : { '$gte' : start, '$lt' : end} }).sort('date', 1)
signals= None
i=0
last_prediction= -500
prev_record_time= None
### Go through events in DB
for event in event_list:
    i+=1
    record_list= event['content']
    keys= None
    ### Go through records in each event
    for record in record_list:
        if record['date']< start or record['date'] > end:
            continue
        if prev_record_time is not None and (record['date']  - prev_record_time).total_seconds() < 5:
            print('Escaped!')
            continue
        else:
            prev_record_time= record['date']
        if 'given2model' not in record:
            if record[prediction_variable] == last_prediction:
                record['given2model']= False
            else:
                last_prediction= record[prediction_variable]
                record['given2model']= True
        keys= record['raw_data'].keys()
        for item in keys:
            record['raw_data'][item]= [record['raw_data'][item]]
        #print ('before',record)
        record['raw_data']['date']=[record['date'] + timedelta(hours= +2)]
        record['raw_data'][output_variable]=[record[output_variable]]
        record['raw_data'][prediction_variable]=[record[prediction_variable]]
        #record['raw_data']['DRZ2']= [float(record['DRZ2'])]
        # record['raw_data']['M8']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M121']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M127']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['M7']= 0#(0 if record['raw_data']['DRZ'][0]<300 else 1)
        # record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0] - record['raw_data']['t_bett'][0], decimals=2)
        # record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]) - record['raw_data']['t_bett'][0], decimals= 2)
        record['raw_data']['t_spindle'][0]= np.round(record['raw_data']['t_spindle'][0], decimals=2)
        record['raw_data']['t_motor']= np.round(float(record['raw_data']['t_motor'][0]), decimals= 2)
        #####Add preprocessed Data##########
        record['raw_data']['0']= record['preprocessed_data'][0]
        record['raw_data']['1']= record['preprocessed_data'][1]
        record['raw_data']['2']= record['preprocessed_data'][2]
        record['raw_data']['3']= record['preprocessed_data'][3]
        record['raw_data']['4']= record['preprocessed_data'][4]
        record['raw_data']['5']= record['preprocessed_data'][5]
        ####################################
        record['raw_data']['given2model']= record['given2model']
        #print ('After ',record)
        lf_signal_point= pd.DataFrame(record['raw_data'])
        if signals is None:
            signals= lf_signal_point
        else:
            signals= signals.append(lf_signal_point,ignore_index= True)
signals['t_motor']= signals['t_motor'].apply(np.float32)
# signals['prediction_abs_error']= signals[prediction_variable] -signals[output_variable]
# signals['prediction_abs_error']= 1000 * signals['prediction_abs_error'].abs()
signals.reset_index(inplace= True)
####smoothing singals####
#signals= smoothing_data(signals,24)
# #signals = remove_peeks(signals, output_variable, 4)
####Apply enterpolation###
#output_variable= str(output_variable+'_smoothed')
upsampled_variable= output_variable#str('interpolated_'+output_variable)
signals[upsampled_variable]= signals[output_variable]
current_welle_z= 1000000
for index, row in signals.iterrows():
    if row[upsampled_variable]!= current_welle_z: ### A New value is available
        current_welle_z= row[upsampled_variable]
    else:
        signals.at[index,upsampled_variable]= np.NaN
signals[upsampled_variable].interpolate(method='linear', order=3, axis= 0,inplace= True)
output=  signals[output_variable]
machine_prediction= signals[prediction_variable]
rough_signals= signals[selected_Columns].to_numpy()
print(rough_signals.shape)

In [ ]:
print('input signal shape=', rough_signals.shape, 'output shape= ', output.shape)

Generate prediction based on the transformed data in DB

Load the Model and Preprocessor

In [ ]:
### load models
pipeline= load(str(preprocessor_dir+preprocessor_name))
print(pipeline)
#pipeline.verbose()
print('preprocessor is loaded successfully')
model= load_model(str(model_dir + model_name), compile= False)
print('Model is loaded successfully')
print ('Model Input= ', model.input_shape,'Model output= ', model.output_shape)
print (model.summary())

Apply preprocessor on the input data

In [ ]:
transformed_signals= pipeline.transform(rough_signals)
transformed_signals.shape

In [ ]:
transformed_signals_windowed, output= generateDataSource(signal_input=transformed_signals, signal_output=output, window= window, shift= shift, sample_rate= sampling_rate)
print('transformed_signals_windowed: ', transformed_signals_windowed.shape, 'label: ', output.shape)

In [ ]:
preprcessed_signals= transformed_signals_windowed#transformed_signals#signals[['0','1','2','3','4','5']].to_numpy()
results= preprcessed_signals
targets= output
output= np.reshape(targets,-1)
print('input: ', results.shape, ' output:', output.shape)

Apply predictions

In [ ]:
preds= model.predict(results)/1000
preds= np.reshape(preds, -1)

In [ ]:
preds_shifted= preds#np.concatenate((np.zeros( shape= (window-1,)), preds))
prediction_error= np.abs(targets- preds_shifted)
min_error= np.min(prediction_error)
max_error= np.max(prediction_error)
mean_error= np.mean(prediction_error)
prediction_error= targets- preds_shifted


In [ ]:
len(preds_shifted)

In [ ]:
input_shift=  (window ) * (shift) #abs(rough_signals.shape[0] - results.shape[0])
print(input_shift)
visualization_df= signals.iloc[input_shift:]
pred_doubled= np.zeros(shape=(len(visualization_df),))
for i in range(0, preds_shifted.shape[0]):
    pred_doubled[i*sampling_rate: i*sampling_rate + sampling_rate ]= preds_shifted[i]
visualization_df['upsampled_predictions']= pred_doubled
visualization_df


labels + prediction +Error and the upper and lower limit in one block

In [ ]:
# scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
# # selected_Columns= ['t_bett','t_motor', 't_spindle' ,'M8', 'M121', 'M127', 'M7', prediction_variable, output_variable, 'db_prediction_abs_error']
# # y_axis_names= selected_Columns
# #print(preds)
# fig= make_subplots(rows=1,cols=1,shared_xaxes= True, print_grid= True, vertical_spacing=0.02)
# ##Verlagerung
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable], name='Gemessene Verlagerung (Welle)', mode= scatter_mode), row= 1, col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[prediction_variable], name='KI-basierte Verlagerung (Welle)', mode= scatter_mode), row= 1, col= 1)

# ##Restfehler
# fig.add_trace(go.Scatter(x= signals['date'], y= 1000 * signals[output_variable] - 1000 * signals[prediction_variable], name=' Restfehler', mode= scatter_mode),  row= 1 , col= 1)
# ## Draw the tolerence +-5
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],5), name='+5 Obere Grenze', mode= scatter_mode),  row= 1 , col= 1)
# fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(signals[output_variable],-5), name='- 5 Obere Grenze', mode= scatter_mode),  row= 1 , col= 1)
# fig.update_yaxes(title_text= 'Verlagerung Micro-Meter', row= 1, col= 1)
# fig.update_layout(height=1200, width=1400, title_text= 'M57002 Machine Data, Versuch am 2023-01-11T20:00:00 ISO 230-3 Trocken')

In [ ]:
## Comparasion between two sources:
from turtle import color

scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
local_model_predictions= 'upsampled_predictions'
#visualization_df[old_model_predictions]=visualization_df[prediction_variable]
selected_Columns= ['t_bett','t_motor', 't_spindle' ,  prediction_variable]
#selected_Columns= ['t_bett','t_motor', 't_spindle','DRZ','DRZ2', prediction_variable]
#selected_Columns.append('prediction_abs_error')
print (selected_Columns)
y_axis_names= selected_Columns
#print(preds)
fig= make_subplots(rows=len(selected_Columns)+1,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)

for i in range(len(selected_Columns)):
    fig.add_trace(go.Scatter(x= visualization_df['date'], y= visualization_df[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1)
    fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
##Draw the prediciton and the real values of displacement on Welle
fig.add_trace(go.Scatter(x= visualization_df['date'],y= visualization_df[local_model_predictions], name=local_model_predictions, mode= scatter_mode),  row= len(selected_Columns) , col= 1)
fig.add_trace(go.Scatter(x= visualization_df['date'],y= visualization_df[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns) , col= 1)
fig.update_yaxes(title_text= output_variable, row= len(selected_Columns), col= 1)
fig.add_trace(go.Scatter(x= visualization_df['date'], y=   1000*(visualization_df[output_variable] - visualization_df[prediction_variable]), name='Mchine Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.add_trace(go.Scatter(x= visualization_df['date'], y=   1000*(visualization_df[output_variable] - visualization_df[local_model_predictions]), name='local Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
## Draw the tolerence +-5
fig.add_trace(go.Scatter(x= visualization_df['date'], y= np.full_like(visualization_df[output_variable],5), name='+5 Max Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.add_trace(go.Scatter(x= visualization_df['date'], y= np.full_like(visualization_df[output_variable],-5), name='-5 Min Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)

#fig.add_trace(go.Scatter(x= signals['date'], y=   1000*( np.abs(signals[output_variable] - signals[prediction_variable])), name='Old Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
fig.update_yaxes(title_text= 'Error (Micro-meter)', row= len(selected_Columns)+1, col= 1)
fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
fig.show()